Movie recomendation system can have multiple approaches. In the following program, I have used Matrix factorisation. The idea is to create a content based recomendation system where we build item profile which gives out  important characters of the item. According to the input given by the user , we come up with a user profile same as the item profile. We specify the user's preference by constructing a utility matrix by assiging values to each item pair. then we conclude by recommending the most suitable items for the user.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


Reading the csv files

In [ ]:
moviesDF = pd.read_csv('/drive/My Drive/Colab Notebooks/movies.csv')
ratingDF = pd.read_csv('/drive/My Drive/Colab Notebooks/ratings.csv')

In [ ]:
moviesDF.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
ratingDF.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


Combining the two tables according to movieId and droping unwanted columns

In [ ]:
ratingDF = ratingDF.drop('timestamp',1)
# the copy here is not necessay but i had made a mistake. To make it up I copied the df into a diffrent variable
movie_genres_df = moviesDF.copy()
movie_genres_df.head(10)

<ipython-input-5-fa69a583e928>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratingDF = ratingDF.drop('timestamp',1)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Removing the year from each movie name

In [ ]:
movie_genres_df['title'] = movie_genres_df.title.str.replace('(\(\d\d\d\d\))', '')
movie_genres_df['title'] = movie_genres_df['title'].apply(lambda x: x.strip())
movie_genres_df.head()

<ipython-input-6-f99864610b31>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movie_genres_df['title'] = movie_genres_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


Making an array out of ever string in the genres by splitting at ever "|" and replacing it in each cell

In [ ]:
movie_genres_df['genres'] = movie_genres_df.genres.str.split("|")
movie_genres_df.head()

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]


We create a item profile of 1s and 0s. If the movie is tagged with a perticular genre then we give the value 1, else 0.

In [ ]:
movie_matrix = movie_genres_df.copy()
for index, row in movie_genres_df.iterrows():
    for genre in row['genres']:
        movie_matrix.at[index, genre] = int(1)
movie_matrix.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",NaN,NaN,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,[Comedy],NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fill the NaN values with 0 to work with the dataset

In [ ]:
movie_matrix = movie_matrix.fillna(0)
movie_matrix

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9739,193585,Flint,[Drama],0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]",0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Lets suppose an user rates the following movies, we take the input as an array with movie and and rating 

In [ ]:
client_input = [
            {'title':'Copycat', 'rating':4.9},
            {'title':'Kids', 'rating':1.0},
            {'title':'Species', 'rating':3.0},
            {'title':"Unzipped", 'rating':2.1},
            {'title':'French Kiss', 'rating':3.5} ]

Convert this array into a dataframe

In [ ]:
input_movies_df = pd.DataFrame(client_input)
display(input_movies_df)

,title,rating
0,Copycat,4.9
1,Kids,1.0
2,Species,3.0
3,Unzipped,2.1
4,French Kiss,3.5


In [ ]:
#compare the titles with the movie_genres_df to get the respective movieId
user_input_movieId = movie_genres_df[movie_genres_df['title'].isin(input_movies_df['title'].tolist())]
client_df = pd.merge(user_input_movieId,input_movies_df)
#droping the unwanted columns and making a user dataframe
client_df = user_input_movieId.drop(['movieId','title'],1)
display(user_input_movieId)
display(client_df)

<ipython-input-12-632b9374cbc4>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  client_df = user_input_movieId.drop(['movieId','title'],1)


,movieId,title,genres
21,22,Copycat,"[Crime, Drama, Horror, Mystery, Thriller]"
147,175,Kids,[Drama]
166,196,Species,"[Horror, Sci-Fi]"
174,206,Unzipped,[Documentary]
202,236,French Kiss,"[Action, Comedy, Romance]"


,genres
21,"[Crime, Drama, Horror, Mystery, Thriller]"
147,[Drama]
166,"[Horror, Sci-Fi]"
174,[Documentary]
202,"[Action, Comedy, Romance]"


Filtering the required user movies from the main dataset and droping movieId, genres, title

In [ ]:
filtered_df = movie_matrix[movie_matrix['movieId'].isin(user_input_movieId['movieId'].tolist())]
filtered_df = filtered_df.drop(['movieId','genres','title'],1)
display(filtered_df)

<ipython-input-13-594b623f647e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  filtered_df = filtered_df.drop(['movieId','genres','title'],1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
21,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
202,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#converting filtered_df into array
filtered_array = filtered_df.to_numpy()
client_ratings = np.array([[4.9,1.0,3.0,2.1,3.5]])
client_ratings.transpose()

# creating Utility profile
# dotproduct of tiltered array and clent ratings 
user_profile = np.dot(client_ratings,filtered_array)

#sum
sum = np.sum(user_profile)

# replacing ever i,j value by its quotient with sum
for i in range(0,1):       #two rows
  for j in range(0,20):    # 21 genres
    user_profile[i][j] = user_profile[i][j]/sum

display(user_profile)

array([[0.        , 0.        , 0.        , 0.07936508, 0.        ,
        0.07936508, 0.13378685, 0.07936508, 0.11111111, 0.11111111,
        0.17913832, 0.11111111, 0.06802721, 0.        , 0.        ,
        0.04761905, 0.        , 0.        , 0.        , 0.        ]])

In [ ]:
user_profile_reshaped = user_profile.reshape(1,20)
movie_matrix_copy = movie_matrix

#storing the movieId in a variable
movieID = movie_matrix[['movieId']]
movie_matrix = movie_matrix.drop(['movieId','title','genres'],axis=1)

# multiplying user profile and transpose of the filtered dataframe

movie_matrix_t = movie_matrix.transpose()
mult = np.matmul(user_profile_reshaped,movie_matrix_t)
mult = mult.transpose()

if [column for column in movieID.columns] not in [column for column in movie_matrix.columns]:
    add = mult.append(movieID)
    add = add[[column for column in movie_matrix.columns if column in add.columns]]

mult['movieId']=movie_matrix_copy['movieId']
mult['title']=movie_matrix_copy['title']
mult.columns=['rating','movieId','title']

# sorting on the basis of their ratings
mult.sort_values(by=['rating'],ascending=False,inplace=True)

#printing top 15 recomendations
top_15=mult.head(15)
top_15 = top_15.drop('movieId',1)
display(top_15)

<ipython-input-15-7575fa145db3>:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  top_15 = top_15.drop('movieId',1)


,rating,title
7441,0.804989,Rubber
4843,0.693878,Ichi the Killer (Koroshiya 1)
6145,0.682540,Pulse
21,0.646259,Copycat
7372,0.614512,Inception
167,0.614512,Strange Days
5556,0.614512,Patlabor: The Movie (Kidô keisatsu patorebâ: T...
5733,0.614512,Gozu (Gokudô kyôfu dai-gekijô: Gozu)
5632,0.614512,Audition (Ôdishon)
3128,0.614512,Manhunter


The movie "Rubber" with a match of 80.4% is recommended to the user followed by other movies in decreasing order of their match.